In [ ]:
# generating Minimum Snap trajectories as a benchmark for the G&CNets
import matplotlib.pyplot as plt
import numpy as np
from sympy import *

# Position and yaw are described by nth order polynomials
t = Symbol('t')

# Degree of polynomials
n=6

# number of laps
n_laps = 10

# number of trajectories
n_traj = 4*n_laps

parameters = [Matrix([['p'+str(i)+str(j)+str(k) for j in range(n+1)] for k in range(4)]) for i in range(n_traj)]
T = [Symbol('T'+str(i)) for i in range(4*n_laps)]

def get_traj(idx):
    P = parameters[idx]
    x   = sum([P[0,i]*t**i for i in range(n+1)])
    y   = sum([P[1,i]*t**i for i in range(n+1)])
    z   = sum([P[2,i]*t**i for i in range(n+1)])
    psi = sum([P[3,i]*t**i for i in range(n+1)])
    return x,y,z,psi

In [ ]:
# Objective function
kr = 4
kpsi = 2

# pos and yaw have equal weight
mur = 1
mupsi = 1

def get_Obj(idx):
    x,y,z,psi = get_traj(idx)
    return integrate(mur*(diff(x,t,kr)**2+diff(y,t,kr)**2+diff(z,t,kr)**2) + mupsi*(diff(psi,t,kpsi)**2), (t, 0, T[idx]))

Obj = sum(get_Obj(i) for i in range(n_traj))

In [ ]:
# Waypoints

d=np.sin(np.pi/4)

wp1 = np.array([4,0,0]) + 0.2*np.array([-d,d,0])
wp2 = np.array([4,3,0]) + 0.2*np.array([-d,-d,0])
wp3 = np.array([0,3,0]) + 0.2*np.array([d,-d,0])
wp4 = np.array([0,0,0]) + 0.2*np.array([d,d,0])

waypoints = np.array([
    [wp1[0],wp2[0],wp3[0],wp4[0]]*n_laps,
    [wp1[1],wp2[1],wp3[1],wp4[1]]*n_laps,
    [0,0,0,0]*n_laps,
    [np.pi/4+i*np.pi/2 for i in range(4*n_laps)]
]).T

In [ ]:
# Get constraints of the ith trajectory
def get_constraints(idx):
    x,y,z,psi = get_traj(idx)
    
    # Equality constraints g(p) = 0
    g=[]
    
    if idx==0:
        # Initial state (hover)
        for i in range(3):
            g.append(diff(x,t,i).subs(t,0))
            g.append(diff(y,t,i).subs(t,0))
            g.append(diff(z,t,i).subs(t,0))
            g.append(diff(psi,t,i).subs(t,0))
    else:
        # Enforce continuity of the first kr derivatives of rT and first kψ derivatives of ψT
        x_,y_,z_,psi_ = get_traj(idx-1)
        for i in range(kr+1):
            g.append(diff(x_,t,i).subs(t,T[idx-1])-diff(x,t,i).subs(t,0))
            g.append(diff(y_,t,i).subs(t,T[idx-1])-diff(y,t,i).subs(t,0))
            g.append(diff(z_,t,i).subs(t,T[idx-1])-diff(z,t,i).subs(t,0))
        for i in range(kpsi+1):
            g.append(diff(psi_,t,i).subs(t,T[idx-1])-diff(psi,t,i).subs(t,0))
    
    # Waypoint Constraints
    xT,yT,zT,psiT = waypoints[idx]
    g.append(x.subs(t,T[idx])-xT)
    g.append(y.subs(t,T[idx])-yT)
    g.append(z.subs(t,T[idx])-zT)
    g.append(psi.subs(t,T[idx])-psiT)
    
    # Velocity Constraints
    g.append(sin(psiT)*diff(x,t).subs(t,T[idx])-cos(psiT)*diff(y,t).subs(t,T[idx]))
    g.append(diff(z,t).subs(t,T[idx]))
    return g

In [ ]:
from amplpy import AMPL

ampl = AMPL()
ampl.eval('option solver snopt;')

ampl.eval('param Tn default '+ str(n_traj) + ';')

# VARIABLES
for t_ in T:
    ampl.eval('var ' + str(t_) + ', >= 0;')
for P in parameters:
    for p in P:
        ampl.eval('var ' + str(p) + ';')

# OBJECTIVE
objective = str(Obj)
objective = objective.replace('**','^')
ampl.eval('minimize myobjective: '+str(objective) +';')

# CONSTRAINTS
ampl.eval('subject to time_constraint: '+str(sum(T))+'=Tn;')

for idx in range(n_traj):
    ampl.eval('subject to time_constraint' + str(idx) + ':' + str(T[idx]) + '= 1.;')

# ADD CONSTRAINTS FOR EACH TRAJ
for idx in range(n_traj):
    g = get_constraints(idx)
    for i in range(len(g)):
        constraint = str(g[i])
        constraint = constraint.replace('**','^')
        ampl.eval('subject to constraint' + str(idx) + '_' + str(i) + ':' + str(constraint) + '=0;')

# SOLVE
ampl.solve()
    
# REMOVE TIME CONSTRIANTS
for idx in range(n_traj):
    ampl.getConstraint('time_constraint'+str(idx)).drop()


ampl.solve()

In [ ]:
# SOLVE AGAIN?
ampl.solve()

In [ ]:
def get_traj_solved(idx):
    x,y,z,psi = get_traj(idx)
    x   = x.subs([(str(p), ampl.getVariable(str(p)).value()) for p in parameters[idx]])
    y   = y.subs([(str(p), ampl.getVariable(str(p)).value()) for p in parameters[idx]])
    z   = z.subs([(str(p), ampl.getVariable(str(p)).value()) for p in parameters[idx]])
    psi = psi.subs([(str(p), ampl.getVariable(str(p)).value()) for p in parameters[idx]])
    return x,y,z,psi

In [ ]:
x,y,z,psi =[],[],[],[]
T_ = np.cumsum([0] + [ampl.getVariable(str(T[i])).value() for i in range(n_traj)])
for i in range(n_traj):
    xi,yi,zi,psii = get_traj_solved(i)
    x.append((xi.subs(t,t-T_[i]),(t>=T_[i])&(t<T_[i+1])))
    y.append((yi.subs(t,t-T_[i]),(t>=T_[i])&(t<T_[i+1])))
    z.append((zi.subs(t,t-T_[i]),(t>=T_[i])&(t<T_[i+1])))
    psi.append((psii.subs(t,t-T_[i]),(t>=T_[i])&(t<T_[i+1])))

x.append((waypoints[-1][0], True))
y.append((waypoints[-1][1], True))
z.append((waypoints[-1][2], True))
psi.append((waypoints[-1][3], True))

x   = Piecewise(*x, evaluate=False)
y   = Piecewise(*y, evaluate=False)
z   = Piecewise(*z, evaluate=False)
psi = Piecewise(*psi, evaluate=False)
print(T_[16])

In [ ]:
import matplotlib as mpl
import matplotlib.cm as cm

norm = mpl.colors.Normalize(0,7)
cmap = cm.jet 

def color_plot(x_axis,y_axis,color_axis,step=1):
    for i in reversed(range(step,len(x_axis),step)):
        ax = plt.gca()
        ax.plot([x_axis[i-step], x_axis[i]],[y_axis[i-step], y_axis[i]], color=cmap(norm(color_axis[i])))
        ax.axis('equal')
        
def color_plot3D(x_axis,y_axis,z_axis,color_axis,step=1):
    ax = plt.axes(projection='3d')
    for i in reversed(range(step,len(x_axis),step)):
        ax.plot3D([x_axis[i-step], x_axis[i]],[y_axis[i-step], y_axis[i]],[z_axis[i-step], z_axis[i]], color=cmap(norm(color_axis[i])))
#         ax.axis('equal')

In [ ]:
vx = diff(x,t)
vy = diff(y,t)
V = sqrt(vx**2+vy**2)

t_ = np.linspace(0,T_[-1],1600)
x_ = lambdify(t,x)
y_ = lambdify(t,y)
z_ = lambdify(t,z)
psi_ = lambdify(t,psi)
V_ = lambdify(t,V)

color_plot(y_(t_),x_(t_),V_(t_))
plt.xlabel('y [m]')
plt.ylabel('x [m]')
plt.grid()
plt.gca().axis('equal')
plt.gcf().colorbar(cm.ScalarMappable(norm=norm, cmap=cmap))
plt.title('Min Snap T=40.0')
plt.show()

In [ ]:
%matplotlib inline
for i in range(n_laps):
    ti = np.linspace(T_[i*4],T_[i*4+4])
    color_plot(y_(ti),x_(ti),V_(ti))
    plt.scatter([0,0,3,3],[0,4,4,0],c='black',zorder=3)
    plt.xlabel('y [m]')
    plt.ylabel('x [m]')
    plt.grid()
    plt.gca().axis('equal')
    plt.gcf().colorbar(cm.ScalarMappable(norm=norm, cmap=cmap))
    plt.title('Lap ' + str(i+1) +': T=' + str(round(T_[i*4+4]-T_[i*4],2)))
    plt.savefig('Figures/MinSnapLap'+str(i+1)+'.png')
    plt.show()

In [ ]:
print(T_[::4])
vx = diff(x,t)
vy = diff(y,t)
vx_ = lambdify(t,vx)
vy_ = lambdify(t,vy)

for i in range(n_laps):
    print('Lap '+str(i))
    print('vx0='+str(vx_(T_[4*i]))+',vxT='+str(vx_(T_[4*i+4])))
    print('vy0='+str(vy_(T_[4*i]))+',vyT='+str(vy_(T_[4*i+4])))

In [ ]:
plt.plot(t_, x_(t_), label='x')
plt.plot(t_, y_(t_))
plt.plot(t_, z_(t_))
plt.plot(t_, psi_(t_))

In [ ]:
%matplotlib
# ax = plt.axes(projection='3d')
color_plot3D(y_(t_),x_(t_),-z_(t_),V_(t_))
wpx = waypoints[0:4][:,0]
wpy = waypoints[0:4][:,1]
wpz = waypoints[0:4][:,2]
plt.gca().scatter3D(wpy,wpx,-wpz,color='black')
plt.gca().set_aspect('equal') 
plt.show()

In [ ]:
lap_times = T_[4::4]-T_[:-4:4]

alpha = np.arange(0.7,1.5,0.05)
table = np.zeros([3, len(alpha)])

for i in range(len(alpha)):
    table[0,i] = alpha[i]
    table[1,i] = T_[16]/alpha[i]
    table[2,i] = T_[-1]/alpha[i]

for i in range(len(alpha)):
    print('|c',end='')
print('|')

for row in table:
    for i in row:
        print(f"{i:.2f}", end='&')
    print('\\'*2)

In [ ]:
z=simplify(z)
psi=simplify(psi)

## Generate C code

In [ ]:
from sympy.utilities.codegen import codegen

# TRANSFORM TO CYBERZOO COORDINATES
x_ = x-2.
y_ = y-1.5
z_ = z-1.

angle = -33*np.pi/180
x_new = x_*cos(angle) - y_*sin(angle)
y_new = x_*sin(angle) + y_*cos(angle)
x_ = x_new
y_ = y_new
psi_ = psi + angle

# TIME SCALE PARAMETER
alpha = Symbol('alpha')
x_ = x_.subs(t,alpha*t)
y_ = y_.subs(t,alpha*t)
z_ = z_.subs(t,alpha*t)
psi_ = psi_.subs(t,alpha*t)

vx_ = diff(x_,t)
vy_ = diff(y_,t)
vz_ = diff(z_,t)

ax_ = diff(vx_,t)
ay_ = diff(vy_,t)
az_ = diff(vz_,t)

print('generating code')
[(c_name, c_code), (h_name, c_header)] = codegen([
    ("get_x", x_),
    ("get_y", y_),
    ("get_z", z_),
    ("get_psi", psi_),
    ("get_vx", vx_),
    ("get_vy", vy_),
    ("get_vz", vz_),
    ("get_ax", ax_),
    ("get_ay", ay_),
    ("get_az", az_)
], "C", "min_snap", header=True, empty=False)

In [ ]:
c_header = c_header.replace('int', 'double')
c_header = c_header.replace('()', '(double alpha, double t)')
# print(c_header)

In [ ]:
c_code = c_code.replace('int', 'double')
c_code = c_code.replace('()', '(double alpha, double t)')
# print(c_code)

In [ ]:
text_file = open('c_code/'+c_name, "w")
n = text_file.write(c_code)
text_file.close()

text_file = open('c_code/'+h_name, "w")
n = text_file.write(c_header)
text_file.close()

In [ ]:
print("cyberzoo waypoint coordinates:")
for i in range(-1,3):
    wpx,wpy = waypoints[i][0:2]
    wpx -= 2
    wpy -= 1.5
    wpx_new = wpx*cos(angle) - wpy*sin(angle)
    wpy_new = wpx*sin(angle) + wpy*cos(angle)
    print('ENU wp'+str((i+1)%4+1), round(wpy_new,2), round(wpx_new,2), 1)

print('')

print("flight plan waypoint definition:")
for i in range(-1,3):
    wpx,wpy = waypoints[i][0:2]
    wpx -= 2
    wpy -= 1.5
    wpx_new = wpx*cos(angle) - wpy*sin(angle)
    wpy_new = wpx*sin(angle) + wpy*cos(angle)
    print('<waypoint name="WP'+str((i+1)%4+1)+'" x="'+str(round(wpy_new,3))+'" y="'+str(round(wpx_new,3))+'" alt="1."/>')

## Test C code

In [ ]:
import subprocess
import ctypes
import os

path = os.path.abspath('c_code')

# https://cu7ious.medium.com/how-to-use-dynamic-libraries-in-c-46a0f9b98270

# Create object files
print('Create object files')
subprocess.call('gcc -fPIC -c *.c', shell=True, cwd=path)

# Create library
print('Create library')
subprocess.call('gcc -shared -Wl,-soname,libtools.so -o libtools.so *.o', shell=True, cwd=path)

# Remove object files
subprocess.call('rm *.o', shell=True, cwd=path)

In [ ]:
lib_path = os.path.abspath("c_code/libtools.so")
fun = ctypes.CDLL(lib_path)

In [ ]:
fun.get_x.argtypes = [ctypes.c_double, ctypes.c_double]
fun.get_x.restype = ctypes.c_double

fun.get_y.argtypes = [ctypes.c_double, ctypes.c_double]
fun.get_y.restype = ctypes.c_double

fun.get_z.argtypes = [ctypes.c_double, ctypes.c_double]
fun.get_z.restype = ctypes.c_double

In [ ]:
x__ = [fun.get_x(1,ti) for ti in t_]
y__ = [fun.get_y(1,ti) for ti in t_]
z__ = [fun.get_z(1,ti) for ti in t_]

In [ ]:
%matplotlib
ax = plt.axes(projection='3d')
ax.plot3D(x__,y__,z__)
plt.show()